In [1]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME               PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-ir-search  GCE       4                                       RUNNING  us-central1-a


In [2]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [3]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage
from pyspark import SparkConf, SparkContext

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *
conf = SparkConf().set("spark.driver.maxResultSize", "10g")
sc = SparkContext.getOrCreate(conf=conf)

In [5]:
spark

In [6]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'assaignment316093632' 
client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    print(b.name)

anchor/0_000.bin
anchor/0_001.bin
anchor/0_002.bin
anchor/0_003.bin
anchor/0_004.bin
anchor/0_005.bin
anchor/0_006.bin
anchor/0_007.bin
anchor/0_posting_locs.pickle
anchor/100_000.bin
anchor/100_001.bin
anchor/100_002.bin
anchor/100_003.bin
anchor/100_004.bin
anchor/100_005.bin
anchor/100_006.bin
anchor/100_007.bin
anchor/100_008.bin
anchor/100_posting_locs.pickle
anchor/101_000.bin
anchor/101_001.bin
anchor/101_002.bin
anchor/101_003.bin
anchor/101_004.bin
anchor/101_005.bin
anchor/101_006.bin
anchor/101_posting_locs.pickle
anchor/102_000.bin
anchor/102_001.bin
anchor/102_002.bin
anchor/102_003.bin
anchor/102_004.bin
anchor/102_005.bin
anchor/102_006.bin
anchor/102_posting_locs.pickle
anchor/103_000.bin
anchor/103_001.bin
anchor/103_002.bin
anchor/103_003.bin
anchor/103_004.bin
anchor/103_005.bin
anchor/103_006.bin
anchor/103_posting_locs.pickle
anchor/104_000.bin
anchor/104_001.bin
anchor/104_002.bin
anchor/104_003.bin
anchor/104_004.bin
anchor/104_005.bin
anchor/104_006.bin
anchor/1

In [7]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [8]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [9]:
from inverted_index_gcp import InvertedIndex

In [10]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
stopwords_frozen = frozenset(stopwords.words('english'))
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def word_count(text, id):
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  entry = []
  listOfUsed = []
  for txt in tokens:
    if((txt not in all_stopwords) and (txt not in listOfUsed)):
      entry.append((txt, (id, tokens.count(txt))))
      listOfUsed.append(txt)
  return entry

def reduce_word_counts(unsorted_pl):
  return sorted(unsorted_pl, key=lambda x: x[0])

def calculate_df(postings):
  return postings.map(lambda x: (x[0], len(x[1])))

def partition_postings_and_write(postings):
  bucket_dict = postings.map(lambda x: (token2bucket_id(x[0]),(x))).groupByKey().mapValues(tuple)
  posting_loc = bucket_dict.map(lambda x:( InvertedIndex.write_a_posting_list((x),bucket_name)))

  return posting_loc

def tokenize(text):
    """
    This function aims in tokenize a text into a list of tokens. Moreover, it filter stopwords.
    
    Parameters:
    -----------
    text: string , represting the text to tokenize.    
    
    Returns:
    -----------
    list of tokens (e.g., list of tokens).
    """
    list_of_tokens =  [token.group() for token in RE_WORD.finditer(text.lower()) if token.group() not in stopwords_frozen]    
    return list_of_tokens



In [15]:
full_path = "gs://wikidata_preprocessed/*"
parquetFile = spark.read.parquet(full_path)
docs = {}

mode = "text"
doc_text_pairs = parquetFile.select(mode, "id")
#.rdd
#pre processing for anchor_text  
if mode == "anchor_text":
  doc_text_pairs = doc_text_pairs.map(lambda x: (' '.join([text for _,text in x[0]]), x[1]))



In [ ]:
#parquetFileTitle = spark.read.parquet(path).select("id", "title")


In [12]:
if mode == "text":
    docss = doc_text_pairs.flatMap(lambda x: [(x[1], tokenize(x[0]))])

In [14]:
print(docss.count())

6348910


In [17]:
for a in doc_text_pairs.take(6348910):
    docs[a[1]] = tokenize(a[0])


22/01/10 11:39:38 ERROR org.apache.spark.scheduler.TaskSetManager: Total size of serialized results of 16 tasks (2.2 GiB) is bigger than spark.driver.maxResultSize (1920.0 MiB)
22/01/10 11:39:38 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 4.0 in stage 11.0 (TID 312) (cluster-ir-search-w-3.c.eng-oven-334519.internal executor 17): TaskKilled (Tasks result size has exceeded maxResultSize)


Py4JJavaError: An error occurred while calling o107.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Total size of serialized results of 16 tasks (2.2 GiB) is bigger than spark.driver.maxResultSize (1920.0 MiB)
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2259)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2208)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2207)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2207)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSetFailed$1(DAGScheduler.scala:1079)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handle

22/01/10 11:39:38 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 19.0 in stage 11.0 (TID 327) (cluster-ir-search-w-1.c.eng-oven-334519.internal executor 21): TaskKilled (Stage cancelled)
22/01/10 11:39:38 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 0.0 in stage 11.0 (TID 308) (cluster-ir-search-w-3.c.eng-oven-334519.internal executor 17): TaskKilled (Stage cancelled)
22/01/10 11:39:38 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 13.0 in stage 11.0 (TID 321) (cluster-ir-search-w-2.c.eng-oven-334519.internal executor 19): TaskKilled (Stage cancelled)
22/01/10 11:39:38 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 14.0 in stage 11.0 (TID 322) (cluster-ir-search-w-2.c.eng-oven-334519.internal executor 19): TaskKilled (Stage cancelled)


In [13]:
docs = docss.collectAsMap()

22/01/10 10:48:08 ERROR org.apache.spark.scheduler.TaskSetManager: Total size of serialized results of 18 tasks (2.1 GiB) is bigger than spark.driver.maxResultSize (1920.0 MiB)
22/01/10 10:48:08 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 19.0 in stage 2.0 (TID 80) (cluster-ir-search-w-3.c.eng-oven-334519.internal executor 9): TaskKilled (Tasks result size has exceeded maxResultSize)
22/01/10 10:48:08 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 24.0 in stage 2.0 (TID 85) (cluster-ir-search-w-1.c.eng-oven-334519.internal executor 12): TaskKilled (Stage cancelled)
22/01/10 10:48:08 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 26.0 in stage 2.0 (TID 87) (cluster-ir-search-w-1.c.eng-oven-334519.internal executor 12): TaskKilled (Stage cancelled)
22/01/10 10:48:08 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 22.0 in stage 2.0 (TID 83) (cluster-ir-search-w-2.c.eng-oven-334519.internal executor 7): TaskKilled (Stage cancelled)
22/01/10 10:

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Total size of serialized results of 18 tasks (2.1 GiB) is bigger than spark.driver.maxResultSize (1920.0 MiB)
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2259)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2208)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2207)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2207)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSetFailed$1(DAGScheduler.scala:1079)
	at org.apache.spark.sch

22/01/10 10:48:08 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 15.0 in stage 2.0 (TID 76) (cluster-ir-search-w-3.c.eng-oven-334519.internal executor 11): TaskKilled (Stage cancelled)
22/01/10 10:48:10 ERROR org.apache.spark.scheduler.TaskSetManager: Total size of serialized results of 19 tasks (2.5 GiB) is bigger than spark.driver.maxResultSize (1920.0 MiB)
22/01/10 10:48:10 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 21.0 in stage 2.0 (TID 82) (cluster-ir-search-w-3.c.eng-oven-334519.internal executor 11): TaskKilled (Tasks result size has exceeded maxResultSize)


In [ ]:
# time the index creation time
t_start = time()
# word counts map
word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
# filtering postings and calculate df
postings_filtered = postings.filter(lambda x: len(x[1])>50)
w2df = calculate_df(postings_filtered)
w2df_dict = w2df.collectAsMap()
# partition posting lists and write out
_ = partition_postings_and_write(postings_filtered).collect()
index_const_time = time() - t_start

In [ ]:
# test index construction time
assert index_const_time < 60*120

In [ ]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)
      

# Create inverted index instance
inverted = InvertedIndex(docs=docs)
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
# write the global stats out
inverted.write_index('.', mode+'_index')

# upload to gs
index_src =  mode+"_index.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp/{index_src}'
!gsutil cp $index_src $index_dst

In [ ]:
!gsutil ls -lh $index_dst

In [ ]:
!gsutil mv gs://assaignment316093632/postings_gcp gs://assaignment316093632/text

In [ ]:
doc_title_pairs = parquetFile.select("id","title").rdd
doc_titels = doc_title_pairs.collectAsMap()
doc_titels_file_name = 'titels.pkl'

with open(doc_titels_file_name, 'wb') as f:
   pickle.dump(doc_titels, f)
doc_titels_dst = f'gs://{bucket_name}/metadata/{doc_titels_file_name}'
!gsutil cp $doc_titels_file_name $doc_titels_dst



In [ ]:
doc_title_pairs = parquetFile.select("id","title","text").rdd
doc_length = doc_title_pairs.map(lambda x : (x[0],len(x[1])+len(x[2]))).collectAsMap()
doc_length_file_name = 'dl.pkl'

with open(doc_length_file_name, 'wb') as f:
   pickle.dump(doc_length, f)
doc_length_dst = f'gs://{bucket_name}/metadata/{doc_length_file_name}'
!gsutil cp $doc_length_file_name $doc_length_dst